In [8]:
############### -------- configure patch size and copy the pathces tuplte to the next cell for computing 1-d averaging for all frames ------------- ###################
%matplotlib widget
import os, shutil, h5py, time
import numpy as np

# specs
frame = 1346
source_file = '2048_B16.h5'                                         

# xy,   width_p, height_p, bounding_box_color for drawing Rectangle
patches = (

( (25, 275), 115, 80, 'red' ),

( (80, 885), 90, 75, 'green' ),

( (735, 240), 55, 50, 'orange'),

( (260, 585), 45, 35, 'blue' ),

( (455, 865), 35, 20, 'pink' ),

)

## semi-specs
valid_range_min, valid_range_max = (0,10)
scattering = '_WAXS2'
masked_file = f'{source_file.split(".")[0]}_masked_{frame}.h5'

## computation
with h5py.File(source_file,'r') as hdf:

    print('data is being loaded ...')
    dset = hdf.get(f'{masked_file.split("_masked")[0]}/primary/data')
    print('data is loaded ...')
    
    print('frame information extraction starts ...')
    dset_saxs = np.expand_dims(dset['pil1M_image'][frame], axis=0)
    dset_waxs = np.expand_dims(dset['pilW2_image'][frame], axis=0)
    print(f'frame information extraction completes with _SAXS shape {dset_saxs.shape} _WAXS shape {dset_waxs.shape}...')

if os.path.isfile(masked_file):
    os.remove(masked_file)
with h5py.File(masked_file,'w') as hdf:

    for frame in range(len(dset_waxs)):
        for xy, width_p, height_p, color in patches:           
            ##### careful here dset_waxs = 0 ; becasue there is only one frame 
            dset_waxs[0][xy[1]: xy[1]+height_p, xy[0]: xy[0]+width_p] = 0   # height px and width px will not be counted img[0:70, 860:860+60]

    dset = hdf.create_group(f'/{masked_file.split("_masked")[0]}/primary/data')
    dset.create_dataset('pil1M_image', data = dset_saxs, compression="lzf")
    dset.create_dataset('pilW2_image', data = dset_waxs, compression="lzf")
    
        
from py4xs.hdf import h5xs,h5exp,lsh5
from py4xs.data2d import Data2d
import numpy as np
import pylab as plt
de = h5exp("exp.h5")
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

dt  = h5xs(masked_file, [de.detectors, qgrid2])
dt.load_data(N=1, debug=True)
#dt.d1s[f'{masked_file.split("_masked")[0]}']['_WAXS2'][0].plot()

# figure with patches
from matplotlib.patches import Rectangle

img = np.clip(dset_waxs[0], valid_range_min, valid_range_max, dtype = np.int8)

plt.figure()
plt.imshow(img, cmap = 'rainbow')
for xy, width_p, height_p, color in patches:
    plt.gca().add_patch(Rectangle(xy=xy, width=width_p, height=height_p,
                                angle=0,
                                edgecolor=color,
                                facecolor='none',
                                fill=False,
                                lw=1))
    
    img[xy[1]: xy[1]+height_p, xy[0]: xy[0]+width_p] = 0   # height px and width px will not be counted img[0:70, 860:860+60]
plt.show()

### plot WAXS data
with h5py.File(masked_file,'r') as hdf:
    dset_merged = hdf.get(f'{masked_file.split("_masked")[0]}/processed')
    dset_merged = dset_merged[scattering][:]

f, ax = plt.subplots(num=f'scattering data')
plt.subplots_adjust(bottom=0.15)
#ax = plt.gca()
ax.set_xlabel("$q (\AA^{-1})$",)
ax.set_ylabel("$I$",)
#ax.set_xscale('log')
#ax.set_yscale('log')
ax.errorbar(qgrid2, dset_merged[0][0], dset_merged[0][1], label=f'{scattering} {frame}')   # here dataset has only one frame
#ax.plot(qgrid2, dset_merged[0][0], label=f'{scattering} {frame}')   # here dataset has only one frame
ax.legend()
plt.show()

if os.path.isfile(masked_file):
    os.remove(masked_file)

data is being loaded ...
data is loaded ...
frame information extraction starts ...
frame information extraction completes with _SAXS shape (1, 1043, 981) _WAXS shape (1, 1043, 981)...
start processing: load_data()
['2048_B16']
processing started: sample = 2048_B16, starting frame = #0
loading data from 2D image:  2048_B16_f00000_SAXS
loading data from 2D image:  2048_B16_f00000_WAXS2
processing completed:  2048_B16 0
writting attribute to 2048_B16: merged
writting attribute to 2048_B16: _SAXS
writting attribute to 2048_B16: _WAXS2
done, time lapsed: 0.46 sec


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
# --------- template matching purposes -------------


# plot zoomed patch figure
#plt.figure()
#pil_im = img[0:150, 800:800+150]
#plt.imshow(pil_im, cmap = 'rainbow', ); 
#plt.show();

# --------- template matching -------------
# template_file = ''
# Cropped image of above dimension OPENCV to PIL_IMAGE
# https://stackoverflow.com/questions/43232813/convert-opencv-image-format-to-pil-image-format
# PIL to OpenCV
#pil_im = Image.fromarray(img_rgb)
#pil_im = pil_im.crop((left, top, right, bottom))
# load template and find height and width of the template shape (convert your BGR image to Gray image)
#template = pil_im
#h, w = template.shape[::]
# print(f'template width {w} Height {h}')
# print(np.min(template), np.max(template), np.min(img), np.max(img))
# print(f'templete {template.shape} type {type(template)} pil_im {img.shape} type {type(img)}')
#res = cv2.matchTemplate(img_rgb, template, cv2.TM_CCOEFF)

In [9]:
# Masked Mica h5py file creation pathes information from the previous cell

## import packages
import os, shutil, h5py, time
import numpy as np

## specs
masked_file = f'{source_file.split(".")[0]}_masked.h5'

## computation
if os.path.isfile(masked_file):
    print(f'DID not PATCH {masked_file} already exists - did not copy, ')
    print(f'1-D Averaging is NOT performed as {masked_file} already exist ')
    print(f'TO do patching on {masked_file} delete the file first manually and run this cell again')


else:
    print(f'{masked_file} is being created in {os.getcwd()} ...')
    shutil.copy2(source_file, masked_file)       # create a new file
    print(f'{masked_file} copy is done')

    # read _WAXS2 source data, replace masked data and save masked out data on _WAXS2
    with h5py.File(masked_file,'r+') as hdf:
        try:
            dset = hdf.get(f'{masked_file.split("_masked")[0]}')
            del dset['processed']
            #del dset["_SAXS"], dset["_WAXS2"], dset["merged"]
        except:
            pass

        dset = dset.get(f'/{masked_file.split("_masked")[0]}/primary/data')
        print(f'{masked_file} Loading data into a numpy array started ')
        dset_waxs = np.array(dset.get('pilW2_image'))
        del dset['pilW2_image']
        print(f'{masked_file} Loading data into a numpy array finished ')


        # Loop over the data to mask out the patches
        print(f'{masked_file} Patching Started ')
        for frame in range(len(dset_waxs)):
            for xy, width_p, height_p, color in patches:
                dset_waxs[frame][xy[1]: xy[1]+height_p, xy[0]: xy[0]+width_p] = 0   # height px and width px will not be counted img[0:70, 860:860+60]
        print(f'{masked_file} Patching Finished')
        
        tic = time.time()
        print(f'{masked_file} file saving staring at {tic}')
        # 'lzf' (no compression_opts), chunks=(1,1043,981), compression_opts=1
        dset.create_dataset('pilW2_image', data = dset_waxs, compression="gzip", compression_opts=1)  
        tac = time.time()
        print(f'{masked_file} file saving finished at {tac-tic} seconds')

    ## create 1d data from lin code
    from py4xs.hdf import h5xs,h5exp,lsh5
    from py4xs.data2d import Data2d
    import numpy as np
    import pylab as plt

    de = h5exp("exp.h5")
    qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

    dt  = h5xs(f'{masked_file}', [de.detectors, qgrid2])
    tic = time.time()
    print(f'Circular averaging staring at {tic}')
    dt.load_data(N=8)
    tac = time.time()
    print(f'total processing time {tac-tic} seconds')


2048_B16_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Dec-15 ...
2048_B16_masked.h5 copy is done
2048_B16_masked.h5 Loading data into a numpy array started 
2048_B16_masked.h5 Loading data into a numpy array finished 
2048_B16_masked.h5 Patching Started 
2048_B16_masked.h5 Patching Finished
2048_B16_masked.h5 file saving staring at 1612543516.6477902
2048_B16_masked.h5 file saving finished at 85.49897980690002 seconds
Circular averaging staring at 1612543602.915169
data received: sn=2048_B16, fr1=0
data received: sn=2048_B16, fr1=465
data received: sn=2048_B16, fr1=930
data received: sn=2048_B16, fr1=1395
data received: sn=2048_B16, fr1=1860
data received: sn=2048_B16, fr1=2325
data received: sn=2048_B16, fr1=2790
data received: sn=2048_B16, fr1=3255
total processing time 428.04442286491394 seconds


In [3]:
########################### ---------- Essential functions ---------- ###########################

### valid indices search function
def h5_top_group(file):
    """ 
        input argument string of a h5py file name
        returns seperator
        seperator = h5_top_group(file)
    """
    return file.split('_masked')[0] if file.find('_masked') >= 0 else file.split('.')[0]

def valid_idx_search(qgrid, Iq):
    """
        Iq shape (#frames, #1-d avg. values) ex. (3721,690)

        function call: 
        idx_l, idx_u, valid_diff_values = valid_idx_search(qgrid2, Iq)
        
        retrun:
            idx_l = first valid   indices    SAXS = 2,   WAXS = 109 , merged = 0
            idx_u = first invalid indices    SAXS = 125, WAXS = 579 , merged = 690
            valid_diff_values = (#frames, only valid values)  ex. 
            '_SAXS'  = (4941, 123) --> 125 - 2 = 123
            '_WAXS2' = (4941, 470) --> 579-109 = 470
    """
    list_ = ~np.isnan(Iq[0,:].flatten())                      # list_ = [False, False, True, True, ...] boolean list in the first frame 
    idx_l = np.argmax(list_==True)                            # find first occurance of a boolean True ; SAXS = 2,   WAXS = 109 , merged = 0
    idx_u  = len(list_) - np.argmax(list_[::-1]==True)        # find last occurance of a invalid number;   SAXS = 125 , WAXS = 579 , merged = 690 valid on 124, 578 , 689 indices
    print(f'low valid idx = {idx_l}, low valid Q = {qgrid[idx_l]:0.3f}, high valid idx = {idx_u-1} , high valid Q = {qgrid[idx_u-1]:0.3f}')
    
    valid_diff_values = Iq[:, idx_l:idx_u]                 # SAXS-Iq --> (1,2:125,:3721)
    
    return idx_l, idx_u, valid_diff_values

### find representative value for each frame
def find_rep_value(qgrid, Iq, args=None, method = 'polyfit'):
    """
    Iq (3721,690) may contain valid or invalid values
    for polynomial averaging:
        method = 'polynomial' 
    for circular averaging:
        method = 'circ'
    diff_patterns = find_rep_value(qgrid2, Iq, args, method = 'polyfit')
    diff_patterns = find_rep_value(qgrid2, Iq, method = 'circ')
    
    """    
    if method == 'polyfit':
        poly_ord, idx_l, idx_u, limit_l, limit_u = args
        n_patterns = len(Iq)
        diff_patterns = [];
        for frame in range(n_patterns):                      # range(100) --> look for first 100 frames

            X = qgrid[idx_l:idx_u]
            y = Iq[frame][idx_l:idx_u]

            # Find coefficients
            coefs = np.polyfit(X,y,poly_ord)
            #print(coefs)

            X_test = X[limit_l:limit_u]
            y_test = y[limit_l:limit_u]
            diff_patterns.append(np.max(y_test - np.polyval(coefs, X_test)));        # (0,2:125,0)  reshape diff_patterns shape (0, 2:125, 3721)
        diff_patterns = np.array(diff_patterns)      
    
    elif method == 'circ':
        n_patterns = len(Iq)
        _ , _ , valid_diff_values = valid_idx_search(qgrid, Iq)
        diff_patterns = np.zeros(len(valid_diff_values));
        
        for frame in range(n_patterns):                      # range(100) --> look for first 100 frames
            diff_patterns[frame] = np.mean(valid_diff_values[frame])

    return diff_patterns

### snaking function
import numpy as np
def snaking( Width, Height, X=None,):
    """
        img_orig = snaking(Width, Height, diff_patterns)
    """
    if X is None:
        X_coor = np.arange(Height*Width).reshape(Height,Width)
    else:
        X_coor = X.reshape(Height,Width)            # input id numpy reshaped as height * width
    img_orig = np.zeros_like(X_coor)                  # 
    temp = np.flipud(X_coor)                        # flipud matrix
    idx_odd = np.arange(1,Height,2)                 # Get odd indices to fliplr
    img_orig[idx_odd,:]  = np.fliplr(temp[idx_odd])   # flipping odd indices
    idx_even = np.arange(0,Height,2)                # Get odd indices to leave to as it is
    img_orig[idx_even,:] = temp[idx_even]             # leaving as it is
    return img_orig

### snaking width_height extraction
## attrs in h5py is recorded in json format

import json
def width_height(file):
    """
        Width, Height = width_height(file)
    """    
    with h5py.File(file,'r') as hdf:
        dset = hdf.get(h5_top_group(file))
        header = json.loads(dset.attrs['start'])
        Height, Width = header['shape']  
    return Width, Height

import h5py
def read_Iq(file, scattering):
    """
        read Iq data
        Iq = read_Iq(file, scattering)
    """
    with h5py.File(file,'r') as hdf:
        Iq = hdf.get(f'{h5_top_group(file)}/processed')              # Iq = hdf.get('2048_B16/processed')
        Iq = np.array(Iq.get(scattering))                            # Iq = np.array(2048_B16/processed/merged')
        Iq = Iq[:,0,:]                                               # Iq shape (3721, 690)

    return Iq

### generate heatmap for differnet scatterings
def plot_heat_map_from_data(img_orig, Width, Height, args, title= None):
    """
        plot_heat_map_from_data(img_orig, Width, Height, args = None, title= None)
    """
    ########## --------- matplotlib mouse hovering function for snaking --------- ########## 
    frame_cor = snaking(Width, Height)                  # snaking indices for heat map, 
    numrows, numcols = Height, Width                    # format_coord function requires this global variables
    def format_coord(x, y):
        col = int(x)                                  # truncate x values
        row = int(y)                                  # truncate y values
        if 0 <= col < numcols and 0 <= row < numrows:
            z = np.flipud(frame_cor)[row, col]        # flipping to get correct value of z     
            return 'x=%1.2f, y=%1.2f, FRAME=%d' % (x, y, z)
        else:
            return 'x=%1.2f, y=%1.2f' % (x, y)        # outside the plotting range, no need
    
    ### plotting
    f, ax = args
    ax.clear()
    ax.imshow(img_orig, cmap = "viridis", interpolation = 'none', origin='upper', extent=[0,Width,0,Height], aspect='equal')
    ax.format_coord = format_coord
    ax.set(title = title, xticks = (np.arange(0,Width,5)), yticks = (np.arange(0,Height,5)))


import pylab as plt
def plot_heat_map_from_file(file, qgrid, scatterings = None, args = None):
    """
        Input args:
            must be tupple scattering = ('_SAXS',)
            args = (f, axs)
        f = plot_heat_map_from_file(file, qgrid, scatterings = scattering)
        f = plot_heat_map_from_file(file, qgrid, scatterings = None, args = (f,axs))
    """
    ## No given scattering argument plot all SAXS, WAXS, Merged
    if scatterings == None:
        scatterings = ('_SAXS',          '_WAXS2',        'merged'    )
    
    f, axs = plt.subplots(1, len(scatterings), num=f'{file}', figsize=(10,5))

    if len(scatterings) == 1:   
        axs = [axs]       # for one scatterings input making axs a list to use in the loop as axs[i]

    ### mouse hovering function call
    Width, Height = width_height(file)
    
    for i, scattering in enumerate(scatterings):
        
        Iq = read_Iq(file, scattering)
        diff_patterns = find_rep_value(qgrid, Iq, method = 'circ')
        img_orig = snaking(Width, Height, diff_patterns)
        
        plot_heat_map_from_data(img_orig, Width, Height, args = (f, axs[i]), title= f'{scattering} {file}')
    #plt.tight_layout()
    plt.show()
    return f

def plot_all_heat_maps_cwd(pdf, qgrid, scatterings):
    """
        import matplotlib.backends.backend_pdf                        # pdf generation package
        pdf = matplotlib.backends.backend_pdf.PdfPages('output.pdf')
        scatterings = ('_SAXS', '_WAXS2')
        
        Function call:
            plot_all_heat_maps_cwd(pdf, scatterings)
    """
    import os
    
    files = []
    for file in [each for each in os.listdir(os.getcwd()) if each.endswith('.h5')]:
        files.append(file)
    files_sorted = sorted(files)

    for file in files_sorted:
        print(f'Loading file {file}')
        
        try:       
            f = plot_heat_map_from_file(file, qgrid, scatterings = scatterings ) # figure object
            print(f'{file} Task Finished. Figure Number = ' , f.number)
        except:    continue
          
        pdf.savefig(f);   #f.savefig("foo.pdf", ) 

    pdf.close();
    print('PDF creation Finished')

In [4]:
%matplotlib widget

### 1d circularly averaged loading for masked file 

# specs  
print(masked_file)
file = masked_file    
scattering = '_WAXS2'
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

# computation
Iq = read_Iq(file, scattering)    
Width, Height = width_height(file)    
n_patterns = Width*Height 

    
import ipywidgets   # from IPython.display import display; display(amp)

f,ax = plt.subplots()
def update_plot(frame):
    ax.clear()
    #ax.set_yscale('log')
    ax.plot(qgrid2, (Iq[frame]), label = f'{scattering} {frame}')

frame = ipywidgets.IntSlider(min=0, max=n_patterns-1, value=0, description = "Frame")
ipywidgets.interactive(update_plot, frame=frame)

2048_B8_masked.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Frame', max=4940), Output()), _dom_classes=('widget-inte…

In [5]:
### show heat map for any file
%matplotlib widget    
file = masked_file    
scatterings = ('_SAXS', '_WAXS2')

#plot_one_heat_maps(file, scatterings)
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)
print('Task Finished. Figure Number = ' , f.number)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
Task Finished. Figure Number =  1


In [6]:
################# --------- Find Q range to plot heat maps --------- #################
%matplotlib widget
import h5py
import pylab as plt
import numpy as np
import ipywidgets as widgets         # from IPython.display import display; display(amp)
from ipywidgets import fixed
#from IPython.display import display  ; display(frame, poly_fit)
# emperical values
# idx_l   = 60 ; idx_u   = 90 ;  limit_l = 13 ; limit_u = 20 ;

# specs
file = masked_file
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])


### q_low, q_high, q_low - idx[2] and q_high + idx[2] for poly fit, number of points
indices = ((0.115, 0.16, 13,   48, '_SAXS' , 'bumpy'),      # 72, 81
           (1.48,  1.57, 10, 4.15, '_WAXS2', 'paraffin'),   # 345, 355
           (1.30,  1.38, 10,  4.7, '_WAXS2', 'amyloid'),    # 309, 323
           (1.89,  1.93,  7,  3.2, '_WAXS2', 'mica')        # 427, 433
          )

############ ------------ computation ------------ ############ 
## matplotlib mouse hovering function for snaking
# format_coord function requires this global variables
Width, Height = width_height(file)
frame_cor = snaking(Width, Height)                  # snaking indices for heat map, 
numrows, numcols = Height, Width                    # format_coord function requires this global variables
n_patterns = Width*Height

# polyfit and q value function
f, axs = plt.subplots(2,len(indices), figsize = (15,9), num = f'{masked_file}')
def update_plot(frame_polyfit, poly_ord = 1):

    for idx_indices,index in enumerate(indices):
        
        ### extracting values from the patches
        idx_l_t = np.max(np.where(qgrid2 <= index[0]))  # 73
        idx_u_t = np.max(np.where(qgrid2 <= index[1]))  # 81
        idx_l =  idx_l_t - index[2]                     # 73 - 13 = 60
        idx_u =  idx_u_t + index[2]                     # 81 + 13 = 90
        limit_l = index[2]
        limit_u = index[2] + idx_u_t - idx_l_t
        scattering = index[4]
        comment = index[5]
        print(idx_l, idx_u, limit_l, limit_u)

        ### read Iq data
        Iq = read_Iq(file, scattering)    
        
        
        X = qgrid2
        y = Iq[frame_polyfit]

        print(idx_l, idx_u, X[idx_l], y[idx_l], X[idx_u-1] , y[idx_u-1])
        X = qgrid2[idx_l:idx_u]
        y = y[idx_l:idx_u]

        coefs = np.polyfit(X,y,poly_ord)
        print(coefs)

        X_test = X[limit_l:limit_u]
        y_test = y[limit_l:limit_u]
        print(X_test, y_test)
        print(y[limit_l + np.argmax(np.abs(y_test - np.polyval(coefs, X_test)))])

        ## plot polynomial and heat map
        axs[0, idx_indices].clear()
        axs[0, idx_indices].scatter(X,y, color='red' , label = 'exp data') 
        axs[0, idx_indices].scatter(X_test, y_test, color='purple' , label = 'fit data')
        axs[0, idx_indices].scatter(X, np.polyval(coefs, X), color = 'black' , label = 'polyfit')
        axs[0, idx_indices].set(title = f'Poly_ord {poly_ord} Frame - {frame_polyfit} {comment}' ,
                                xlabel = 'X' , ylabel = 'y', xscale='linear', yscale = 'linear' )
        axs[0, idx_indices].legend()

        ## gather heat map data
        args = poly_ord, idx_l, idx_u, limit_l, limit_u
        diff_patterns = find_rep_value(qgrid2, Iq, args, method = 'polyfit')
        
        # snaked diff. patterns and plot heat map
        img_orig = snaking(Width, Height, diff_patterns)   
        plot_heat_map_from_data(img_orig, Width, Height, args = (f,axs[1, idx_indices]), title = (f'Q~{index[0], index[1]} {scattering} {index[3]}$\AA$'), )

#        axs[1, idx_indices].set_yticklabels([str(i) for i in [*range(0,Height,3)][::-1]], fontsize=8)        
#        f.colorbar(pos, ax= axs[1, idx_indices])
    
    plt.suptitle('Polyfit and Heat maps')
    plt.tight_layout()
    plt.show()

frame = widgets.IntSlider(min=0, max=n_patterns-1, value=0, description="Frame")
poly_ord = widgets.IntSlider(min=1, max=10, value=1, description='Poly fit order')
widgets.interactive(update_plot, frame_polyfit=frame, poly_ord = poly_ord)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Frame', max=4940), IntSlider(value=1, description='Poly …

In [9]:
%matplotlib widget
from py4xs.hdf import h5xs
import os
from py4xs.slnxs import trans_mode

print(f'{os.getcwd()}')

# specs
# file = '1934_SN_masked.h5'  
# frames = (4518, 3687, 3045)     # amyloid, mica, dark  ---> frames can be choosen from SAXS heat map

file = '2048_B8_masked.h5'
frames = (2080, 1358, 1824)     # amyloid, mica, dark
show_eb = False                  # show errorbar in the plot


## semi-spec
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

# creating objects
dt  = h5xs(f'{file}', transField="em2_sum_all_mean_value")
sn = h5_top_group(file)
dt.load_d1s(sn=sn)


## heat map from file
scatterings = ('_SAXS', '_WAXS2')
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)

### 1-d plot for bright, moderate and dark frames
scatterings = ('_SAXS',          '_WAXS2',        'merged',    )
colors =      (('r','b','g'), ('r','b','g'), ('r+','bo','g--'),)
f, axs = plt.subplots(1, 3, figsize=(15,5), num=f'{file}')
xscale = 'linear'
yscale = 'log'

for i, (scattering, color) in enumerate(zip(scatterings,colors)):
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[0]].data, color[0], label = f'bright {frames[0]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[1]].data, color[1], label = f'moderate {frames[1]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[2]].data, color[2], label = f'dark {frames[2]}')
    axs[i].set(title = f'{scattering} {file}', xscale= xscale, yscale= yscale)
    axs[i].legend()

## background correction calculations
sc_factor=1.

dset = dt.d1s[sn][scattering][frames[0]]
dbak = dt.d1s[sn][scattering][frames[2]]

dt.set_trans(transMode=trans_mode.external)
sc  = dset.trans / dbak.trans
print(f'Scaling factor, {dset.trans}/{dbak.trans} = {sc}')
Iq  = dset.data - dbak.data * sc * sc_factor
dI  = dset.err  + dbak.err * sc * sc_factor
idx = (dset.data > 0) & (dbak.data > 0)

## plotting settings
f, axs = plt.subplots(1,2,figsize=(16,4))
plt.subplots_adjust(bottom=0.15)

axs[0].plot(dbak.qgrid[idx], dset.data[idx], color[0], label = f'bright {frames[0]}')
axs[0].plot(dbak.qgrid[idx], dbak.data[idx], color[2], label = f'dark-bcg {frames[2]}')

if show_eb:
    axs[0].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx])
    axs[1].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx], label = f'{frames[0]} bcor')
else:
    axs[0].plot(dbak.qgrid[idx], Iq[idx], label = 'bcor')
    axs[1].plot(dbak.qgrid[idx], Iq[idx], label = f'{frames[0]} bcor')


axs[0].set(xlabel = "$q (\AA^{-1})$", ylabel = "$I$", title = f'Background correction {file} errorbar = {show_eb}', xscale = xscale, yscale = yscale, )
axs[1].set(xscale = xscale, yscale = yscale, title = f'Final Background Subtracted {file} errorbar = {show_eb}' , )
axs[0].legend()
axs[1].legend()


/Users/bashit.a/Documents/Alzheimer/Dec-15


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'NoneType' object has no attribute 'get'

In [59]:
# pdf generation package all files in the cwd

%matplotlib widget
import pylab as plt
import matplotlib.backends.backend_pdf                        
import h5py
import numpy as np


### heat plots for all files in a folder
pdf = matplotlib.backends.backend_pdf.PdfPages('output.pdf')
scatterings = ('_SAXS', '_WAXS2')
plot_all_heat_maps_cwd(pdf, qgrid2, scatterings)

Loading file 1934_B8.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
1934_B8.h5 Task Finished. Figure Number =  1
Loading file 1934_SN.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
1934_SN.h5 Task Finished. Figure Number =  2
Loading file 1934_SN_masked.h5


...

Loading file mica.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
mica.h5 Task Finished. Figure Number =  19
Loading file std.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

PDF creation Finished


In [10]:
# import pylab as plt
# from matplotlib.widgets import Cursor
# import numpy as np
# x,  y  = np.random.randn(2,10)
# print(x.shape, y.shape)
# f, axs = plt.subplots(1,2)
# axs[0].plot(x,y,'o')
# axs[1].plot(x,y,'o')

# cursor = Cursor(axs[0],
#                 horizOn = True,
#                 vertOn = True,
#                 color= 'green',
#                 linewidth = 1
#                )
# pos = []
# def onclick(event):
#     x1, y1 = event.xdata, event.ydata
#     pos.append((x1,y1))
#     print(x1,y1)

# f.canvas.mpl_connect('button_press_event',  onclick)
#print(pos)

# def hover(event):
#     if event.inaxes != ax:
#         return
#     info = 'x={:.2f}, y={:.2f}'.format(event.xdata, event.ydata)
#     ax.format_coord = lambda x, y: info
# cid = ax.figure.canvas.mpl_connect("motion_notify_event", hover)

# ratio = Height/Width
# xleft, xright = ax.get_xlim()
# ybottom, ytop = ax.get_ylim()
# abs((xright-xleft)/(ybottom-ytop))*ratio
# 1/ax.get_data_ratio()

In [11]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
# Fixing random state for reproducibility
np.random.seed(19680801)

Height = 3
Width = 6
X = np.random.randint(1, 20, size=(Height, Width))

fig, ax = plt.subplots()
ax.imshow(X, interpolation = 'none', origin='upper', extent=[0,Width,0,Height],aspect='equal')

def snaking( Width, Height, X=None,):
    if X is None:
        X_coor = np.arange(Height*Width).reshape(Height,Width)
    else:
        X_coor = X
    output = np.zeros_like(X_coor)           # 
    temp = np.flipud(X_coor)                # flipud matrix
    idx_odd = np.arange(1,Height,2)                # Get odd indices to fliplr
    output[idx_odd,:]  = np.fliplr(temp[idx_odd])   # flipping odd indices
    idx_even = np.arange(0,Height,2)               # Get odd indices to leave to as it is
    output[idx_even,:] = temp[idx_even]             # leaving as it is
    return output

frame_cor = snaking(Width, Height)
diff_patterns = snaking(Width, Height, X)

print('Diff_patt_idx_before snaking: \n', np.arange(Height*Width).reshape(Height,Width), '\n\n',
      'Diff_patt_before snaking: \n', X, '\n\n', 
      'X,Y coordinates_after_snaking: \n', frame_cor, '\n\n',
      'Diff_patt_after snaking: \n', diff_patterns, '\n\n',)

numrows, numcols = X.shape
def format_coord(x, y):
    col = int(x)
    row = int(y)
    if 0 <= col < numcols and 0 <= row < numrows:
        z = np.flipud(frame_cor)[row, col]
        return 'x=%1.4f, y=%1.4f, FRAME=%d' % (x, y, z)
    else:
        return 'x=%1.4f, y=%1.4f' % (x, y)

ax.format_coord = format_coord

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Diff_patt_idx_before snaking: 
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]] 

 Diff_patt_before snaking: 
 [[13 14  7  3  3  4]
 [18  5 11 19 12  9]
 [ 6 12  4  6 17 10]] 

 X,Y coordinates_after_snaking: 
 [[12 13 14 15 16 17]
 [11 10  9  8  7  6]
 [ 0  1  2  3  4  5]] 

 Diff_patt_after snaking: 
 [[ 6 12  4  6 17 10]
 [ 9 12 19 11  5 18]
 [13 14  7  3  3  4]] 


